In [7]:
import tsplib95 
import networkx as nx
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from itertools import combinations
warnings.filterwarnings("ignore", category=Warning)
# import matplotlib.pyplot as plt
# import numpy as np
problem = tsplib95.load('dantzig42.tsp')
# print(G.edges[1,4])
# print(min(G,edges))
G = problem.get_graph()
# nx.draw_networkx(G,pos=nodePos)
# n = 42
import time

start_time = time.time()


In [8]:
pos = problem.display_data
print(pos)
for n, p in pos.items():
    G.nodes[n]['pos'] = p

{1: [170.0, 85.0], 2: [166.0, 88.0], 3: [133.0, 73.0], 4: [140.0, 70.0], 5: [142.0, 55.0], 6: [126.0, 53.0], 7: [125.0, 60.0], 8: [119.0, 68.0], 9: [117.0, 74.0], 10: [99.0, 83.0], 11: [73.0, 79.0], 12: [72.0, 91.0], 13: [37.0, 94.0], 14: [6.0, 106.0], 15: [3.0, 97.0], 16: [21.0, 82.0], 17: [33.0, 67.0], 18: [4.0, 66.0], 19: [3.0, 42.0], 20: [27.0, 33.0], 21: [52.0, 41.0], 22: [57.0, 59.0], 23: [58.0, 66.0], 24: [88.0, 65.0], 25: [99.0, 67.0], 26: [95.0, 55.0], 27: [89.0, 55.0], 28: [83.0, 38.0], 29: [85.0, 25.0], 30: [104.0, 35.0], 31: [112.0, 37.0], 32: [112.0, 24.0], 33: [113.0, 13.0], 34: [125.0, 30.0], 35: [135.0, 32.0], 36: [147.0, 18.0], 37: [147.5, 36.0], 38: [154.5, 45.0], 39: [157.0, 54.0], 40: [158.0, 61.0], 41: [172.0, 82.0], 42: [174.0, 87.0]}


In [11]:
def calculate_tour_distance(tour):
    total_distance = 0
    for i in range(len(tour)):
        v1 = tour[i]
        v1p1 = tour[(i + 1) % len(tour)]
        edge_dist = problem.get_weight(v1,v1p1)
        total_distance += edge_dist
    return total_distance
def has_duplicates(lst):
    return len(lst) != len(set(lst))

In [1]:
tour = [1, 41, 42, 2, 40, 39, 38, 37, 35, 34, 31, 30, 32, 33, 29, 28, 27, 26, 25, 24, 10, 9, 8, 7, 6, 5, 4, 3, 36, 21, 22, 23, 17, 16, 18, 19, 20, 13, 14, 15, 12, 11]

In [4]:
tour = [8, 5, 32, 27, 1, 37, 29, 10, 15, 22, 12, 21, 20, 19, 18, 16, 13, 17]

In [9]:
import pandas as pd

def find_worst_edge(tour):
    seq_best_edge = pd.DataFrame(columns=['V1', 'V1+1', 'Distance'])
    
    for i in range(len(tour)):
        v1 = tour[i]
        v1p1 = tour[(i + 1) % len(tour)]
        edge_dist = problem.get_weight(v1, v1p1)
        seq_best_edge = seq_best_edge.append({'V1': v1, 'V1+1': v1p1, 'Distance': edge_dist}, ignore_index=True)
        
    seq_best_edge = seq_best_edge.sort_values('Distance', ascending=False).reset_index(drop=True)
    
    for i in range(len(seq_best_edge)):
        edge_start = seq_best_edge['V1'][i]
        edge_end = seq_best_edge['V1+1'][i]
        worst_edge_index = tour.index(edge_start)
        
        for j in range(len(tour)):
            if j > worst_edge_index and j != (worst_edge_index + 1) % len(tour):
                for k in range(j + 2, len(tour)):
                    if k != (worst_edge_index + 1) % len(tour) and (k + 1) % len(tour) != worst_edge_index:
                        old_dist = calculate_tour_distance(tour)
                        opt = pd.DataFrame(columns=['tour', 'type', 'dis'])
                        opt = opt.append({'tour': tour, 'type': 'O', 'dis': old_dist}, ignore_index=True)
                        print("old", old_dist)
                        
                        new_tour1 = tour[:worst_edge_index+1] + tour[worst_edge_index + 1: j + 1][::-1]+tour[(j + 1) % len(tour): k + 1][::-1]+tour[k+1:]
                        new_dist1 = calculate_tour_distance(new_tour1);
                        opt = opt.append({'tour': new_tour1, 'type': '1', 'dis': new_dist1}, ignore_index=True)
                                
                        new_tour2 = tour[:worst_edge_index+1]+tour[(j + 1) % len(tour): k + 1]+tour[worst_edge_index + 1: j + 1]+tour[k+1:]
                        new_dist2 = calculate_tour_distance(new_tour2)
                        opt = opt.append({'tour': new_tour2, 'type': '2', 'dis': new_dist2}, ignore_index=True)
                        
                        new_tour3 = tour[:worst_edge_index+1]+tour[(j + 1) % len(tour): k + 1]+tour[worst_edge_index + 1: j + 1][::-1]+tour[k+1:]
                        new_dist3 = calculate_tour_distance(new_tour3)
                        opt = opt.append({'tour': new_tour3, 'type': '3', 'dis': new_dist3}, ignore_index=True)
                        
                        new_tour4 = tour[:]+tour[(j + 1) % len(tour): k + 1][::-1]+tour[worst_edge_index + 1: j + 1]+tour[k+1:]
                        new_dist4 = calculate_tour_distance(new_tour4)
                        opt = opt.append({'tour': new_tour4, 'type': '4', 'dis': new_dist4}, ignore_index=True)
                        
                        opt = opt.sort_values('dis', ascending=True).reset_index(drop=True)
                                
                        if opt['dis'][0] < old_dist and has_duplicates(opt['tour'][0])== False:
                            print("newd", opt['dis'][0], opt['tour'][0])
                            return opt['tour'][0]
    print("oldd", old_dist)                                              
    return tour

In [12]:
def process(tour):
    
    while True:
        print(1)
        new_tour= find_worst_edge(tour)
        
        if new_tour == tour:
            distance = sum(problem.get_weight(new_tour[i], new_tour[
                
                i + 1]) for i in range(len(new_tour) - 1))
            return tour, distance
        else:
            tour = new_tour
            
#             print("processde",tour_dist)
            continue
            
        
tour, distance = process(tour)
end_time = time.time()
print(f"Runtime of the code is {end_time - start_time} seconds")
print("Best tour:", tour)
print("Distance:", calculate_tour_distance(tour))


1
old 927
newd 879 [8, 5, 32, 27, 1, 37, 29, 10, 12, 21, 22, 15, 20, 19, 18, 16, 13, 17]
1
old 879
old 879
newd 871 [8, 5, 32, 27, 37, 1, 12, 10, 29, 21, 22, 15, 20, 19, 18, 16, 13, 17]
1
old 871
old 871
old 871
newd 848 [8, 5, 32, 27, 37, 1, 10, 12, 15, 22, 21, 29, 20, 19, 18, 16, 13, 17]
1
old 848
old 848
old 848
old 848
old 848
old 848
old 848
old 848
old 848
old 848
old 848
old 848
old 848
old 848
old 848
old 848
old 848
newd 794 [8, 5, 32, 27, 37, 1, 22, 21, 29, 20, 19, 18, 16, 13, 17, 15, 12, 10]
1
old 794
old 794
old 794
old 794
old 794
old 794
old 794
old 794
old 794
old 794
old 794
old 794
old 794
newd 788 [8, 5, 32, 27, 37, 1, 29, 21, 22, 13, 16, 18, 19, 20, 17, 15, 12, 10]
1
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
old 788
o

old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575
old 575


In [3]:
import tsplib95 
import networkx as nx
import pandas as pd


def calculate_tour_distance(tour, problem):
    total_distance = 0
    for i in range(len(tour)):
        v1 = tour[i]
        v1p1 = tour[(i + 1) % len(tour)]
        edge_dist = problem.get_weight(v1,v1p1)
        total_distance += edge_dist
    return total_distance


def find_worst_edge(tour, problem):
    seq_best_edge = pd.DataFrame(columns=['V1', 'V1+1', 'Distance'])
    
    for i in range(len(tour)):
        v1 = tour[i]
        v1p1 = tour[(i + 1) % len(tour)]
        edge_dist = problem.get_weight(v1, v1p1)
        new_row = pd.DataFrame([{'V1': v1, 'V1+1': v1p1, 'Distance': edge_dist}])
        seq_best_edge = pd.concat([seq_best_edge, new_row], ignore_index=True)
        
    seq_best_edge = seq_best_edge.sort_values('Distance', ascending=False).reset_index(drop=True)
    
    for i in range(len(seq_best_edge)):
        edge_start = seq_best_edge['V1'][i]
        edge_end = seq_best_edge['V1+1'][i]
        worst_edge_index = tour.index(edge_start)
        
        for j in range(len(tour)):
            if j > worst_edge_index and j != (worst_edge_index + 1) % len(tour):
                for k in range(j + 2, len(tour)):
                    if k != (worst_edge_index + 1) % len(tour) and (k + 1) % len(tour) != worst_edge_index:
                        old_dist = calculate_tour_distance(tour, problem)
                        opt = pd.DataFrame(columns=['tour', 'type', 'dis'])
                        new_row_opt = pd.DataFrame([{'tour': tour, 'type': 'O', 'dis': old_dist}])
                        opt = pd.concat([opt, new_row_opt], ignore_index=True)
                        # print("old", old_dist)
                        
                        new_tour1 = tour[:worst_edge_index+1] + tour[worst_edge_index + 1: j + 1][::-1]+tour[(j + 1) % len(tour): k + 1][::-1]+tour[k+1:]
                        new_dist1 = calculate_tour_distance(new_tour1, problem);
                        new_row1 = pd.DataFrame([{'tour': new_tour1, 'type': '1', 'dis': new_dist1}])
                        opt = pd.concat([opt, new_row1], ignore_index=True)

                                
                        new_tour2 = tour[:worst_edge_index+1]+tour[(j + 1) % len(tour): k + 1]+tour[worst_edge_index + 1: j + 1]+tour[k+1:]
                        new_dist2 = calculate_tour_distance(new_tour2, problem)
                        new_row2 = pd.DataFrame([{'tour': new_tour2, 'type': '2', 'dis': new_dist2}])
                        opt = pd.concat([opt, new_row2], ignore_index=True)
                        
                        new_tour3 = tour[:worst_edge_index+1]+tour[(j + 1) % len(tour): k + 1]+tour[worst_edge_index + 1: j + 1][::-1]+tour[k+1:]
                        new_dist3 = calculate_tour_distance(new_tour3, problem)
                        new_row3 = pd.DataFrame([{'tour': new_tour3, 'type': '3', 'dis': new_dist3}])
                        opt = pd.concat([opt, new_row3], ignore_index=True)

                        
                        new_tour4 = tour[:]+tour[(j + 1) % len(tour): k + 1][::-1]+tour[worst_edge_index + 1: j + 1]+tour[k+1:]
                        new_dist4 = calculate_tour_distance(new_tour4, problem)
                        new_row4 = pd.DataFrame([{'tour': new_tour4, 'type': '4', 'dis': new_dist4}])
                        opt = pd.concat([opt, new_row4], ignore_index=True)
                        
                        opt = opt.sort_values('dis', ascending=True).reset_index(drop=True)
                                
                        if opt['dis'][0] < old_dist and len(set(opt['tour'][0])) == len(opt['tour'][0]):
                            return opt['tour'][0]

    # print("oldd", old_dist)                                              
    return tour
def three_opt(tour, problem):
    improvement = True
    while improvement:
        improvement = False
        for i in range(len(tour) - 2):
            for j in range(i+2, len(tour)):
                for k in range(j+2, len(tour) + (i-0)):
                    if k >= len(tour): # wrap around
                        k -= len(tour)
                    if i != 0 or k != len(tour)-1:
                        new_tour = tour[:]
                        new_tour[i+1:j+1] = reversed(tour[i+1:j+1])
                        new_tour[j+1:k+1] = reversed(tour[j+1:k+1])
                        if calculate_tour_distance(new_tour, problem) < calculate_tour_distance(tour, problem):
                            tour = new_tour
                            improvement = True
    return tour, calculate_tour_distance(tour, problem)
print(tour)

[1, 41, 42, 2, 40, 39, 38, 37, 35, 34, 31, 30, 32, 33, 29, 28, 27, 26, 25, 24, 10, 9, 8, 7, 6, 5, 4, 3, 36, 21, 22, 23, 17, 16, 18, 19, 20, 13, 14, 15, 12, 11]


Greedy
Best tour: [1, 41, 40, 39, 38, 37, 35, 36, 34, 33, 32, 31, 30, 29, 28, 21, 20, 19, 18, 16, 14, 15, 13, 17, 22, 23, 11, 12, 10, 25, 24, 27, 26, 9, 8, 7, 6, 5, 4, 3, 2, 42]
Distance: 726

Nearest neighbour
Best tour: [1, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 21, 20, 19, 18, 16, 15, 14, 13, 17, 22, 23, 12, 11, 24, 27, 26, 25, 10, 9, 8, 3, 7, 6, 5, 4, 42, 2]
Distance: 708

Nearest Insertion
Best tour: [1, 42, 2, 38, 37, 36, 35, 34, 33, 32, 29, 30, 31, 6, 7, 8, 25, 24, 26, 27, 28, 21, 20, 19, 18, 15, 14, 16, 13, 17, 22, 23, 11, 12, 10, 9, 3, 4, 5, 39, 40, 41]
Distance: 764